In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator
import pandas as pd
import numpy as np
from surprise import Dataset, SVD
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator
from tqdm import tqdm
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.preprocessing import MinMaxScaler
import random

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [42]:
df_info

,index,id,title,is_paid,price,headline,num_subscribers,avg_rating,num_reviews,num_comments,...,content_length_min,published_time,last_update_date,category,subcategory,topic,language,course_url,instructor_name,instructor_url
0,1766,84482.0,Ovation Public Speaking - Speaking Methods Mas...,True,59.99,Public speaking is a crucial skill to have in ...,4.0,0.0,0.0,0.0,...,234.0,2022-05-17,2022-05-16,Business,Communication,Public Speaking,English,/course/ovation-public-speaking-speaking-metho...,Ovation Public Speaking,/user/nickcunningham2/
1,1768,84660.0,Method Lesson 4: Speaking with Modulation,True,19.99,Speaking Method #4 focuses on Modulation. Thi...,418.0,3.0,1.0,1.0,...,37.0,2022-04-03,2022-04-02,Business,Communication,Public Speaking,English,/course/method-lesson-4-speaking-with-modulation/,Ovation Public Speaking,/user/nickcunningham2/
2,1778,84664.0,Method Lesson 5: Speaking with Enthusiasm,True,19.99,Lesson 5 focuses on Enthusiasm. This course w...,522.0,4.8,5.0,1.0,...,33.0,2022-04-15,2022-04-14,Personal Development,Career Development,Public Speaking,English,/course/method-lesson-5-speaking-with-enthusiasm/,Ovation Public Speaking,/user/nickcunningham2/
3,1780,84626.0,Method Lesson 3: Speaking with Power,True,24.99,Method Lesson 3 focuses on using power and int...,0.0,0.0,0.0,0.0,...,45.0,2022-05-16,2022-05-15,Personal Development,Career Development,Public Speaking,English,/course/method-lesson-3-speaking-with-power/,Ovation Public Speaking,/user/nickcunningham2/
4,1790,85752.0,Content Lesson 2: Subject Development,True,19.99,Content Lesson 2 focuses on Subject Developmen...,0.0,0.0,0.0,0.0,...,48.0,2022-06-19,2022-06-18,Business,Communication,Presentation Skills,English,/course/content-lesson-2-subject-development/,Ovation Public Speaking,/user/nickcunningham2/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36119,209729,4913954.0,Let's Speak Urdu - The Urdu Grammar,True,19.99,Urdu - Become fluent in this beautiful South A...,3.0,0.0,0.0,0.0,...,54.0,2022-10-05,2022-10-06,Teaching & Academics,Language Learning,Urdu Language,English,/course/lets-speak-urdu-the-grammar/,Jawaid Hameed,/user/jawaid-hameed/
36120,209730,4914146.0,CompTIA Linux+ (XKO-004/005 # 2 Practice Exam ...,True,49.99,Practice Latest exam questions with detailed e...,0.0,0.0,0.0,0.0,...,0.0,2022-10-05,2022-10-05,IT & Software,IT Certifications,CompTIA Linux+,English,/course/comptia-linux-xko-004005-2-practice-ex...,Jean-François d'Halluin,/user/badre-lini/
36121,209731,4914002.0,CISSP 4 full exams #1 : All CISSP domains - 12...,True,49.99,Practice Latest exam questions with detailed e...,0.0,0.0,0.0,0.0,...,0.0,2022-10-05,2022-10-05,IT & Software,IT Certifications,CISSP - Certified Information Systems Security...,English,/course/cissp-4-full-exams-1-all-cissp-domains...,Jean-François d'Halluin,/user/badre-lini/
36122,209732,4913934.0,JD Edwards EnterpriseOne Fixed Assets Accounti...,True,119.99,Full-length course (Part-3 of 3) on JD Edwards...,0.0,0.0,0.0,0.0,...,82.0,2022-10-05,2022-10-05,Finance & Accounting,Other Finance & Accounting,Financial Accounting,English,/course/jde-fixed-assets-accounting-part-3/,Niranjan Bhatia,/user/systemic-software-solutions/


In [2]:
df_info = pd.read_csv('info2017.csv')
df_comment = pd.read_csv('comment2017.csv')

In [6]:
english_courses = df_info.copy()

df_comment['date'] = pd.to_datetime(df_comment['date'], errors='coerce')
df_comment_2022 = df_comment[df_comment['date'].dt.year == 2022]

comment_counts = df_comment_2022['display_name'].value_counts()
valid_users = comment_counts[(comment_counts >= 5) & (comment_counts <= 100)].index
df_comment_filtered = df_comment_2022[df_comment_2022['display_name'].isin(valid_users)]

# Удаляем неанглийские комментарии на основе ascii
df_comment_filtered = df_comment_filtered[df_comment_filtered['comment'].apply(lambda x: isinstance(x, str) and x.isascii())]

# Оставим курсы, которые есть в `english_courses`
df_comment_filtered = df_comment_filtered[df_comment_filtered['course_id'].isin(english_courses['id'])]


# Шаг 1: Найти топ-40 курсов по количеству комментариев
top_40_courses = (
    df_comment_filtered['course_id']
    .value_counts()
    .sample(35)
    .index
)

# Шаг 2: Отфильтровать комментарии только по этим курсам
filtered_by_top_courses = df_comment_filtered[df_comment_filtered['course_id'].isin(top_40_courses)]

# Шаг 3: Найти топ-500 комментаторов по этим курсам
top_500_users = (
    filtered_by_top_courses['display_name']
    .value_counts()
    .head(10000)
    .index
)

# Шаг 4: Финальный фильтр — только комментарии от топ-500 пользователей по топ-40 курсам
df_top_comments = filtered_by_top_courses[
    filtered_by_top_courses['display_name'].isin(top_500_users)
]

# Заменяем df_comment_filtered на финальный отфильтрованный набор
df_comment_filtered = df_top_comments.reset_index(drop=True)

# Шаг 1: Пересоздаем df_info_filtered ТОЛЬКО сейчас, пока course_id ещё НЕ изменён
df_info_filtered = english_courses[english_courses['id'].isin(df_comment_filtered['course_id'])].reset_index(drop=True)

# Шаг 2: Обновляем индексы
df_comment_filtered.index += 1
df_info_filtered.index += 1

# Шаг 3: Уникальные пользователи и курсы — только из отфильтрованного набора
user_mapping = {name: idx for idx, name in enumerate(df_comment_filtered['display_name'].unique(), start=1)}
course_mapping = {cid: idx for idx, cid in enumerate(df_info_filtered['id'].unique(), start=1)}

# Шаг 4: Применяем маппинг
df_comment_filtered['user_id'] = df_comment_filtered['display_name'].map(user_mapping)
df_comment_filtered['course_id'] = df_comment_filtered['course_id'].map(course_mapping)

df_info_filtered['course_id'] = df_info_filtered['id'].map(course_mapping)

# Финальные таблицы
df_comment_final = df_comment_filtered.drop(columns=['display_name', 'index', 'id'], errors='ignore')
df_info_final = df_info_filtered.drop(columns=['id', 'index', 'language', 'instructor_url'], errors='ignore')
df_comment_final['rate'] = df_comment_final['rate'].round()

# Проверим
print("Уникальных курсов:", df_info_final['course_id'].nunique())  # Должно быть 40
print("Уникальных пользователей:", df_comment_final['user_id'].nunique())  # Должно быть <= 500

Уникальных курсов: 35
Уникальных пользователей: 160


In [7]:
courses = df_info_final.copy()
users = df_comment_final.copy()
df_ratings = df_comment_final[['user_id', 'course_id', 'rate', 'date']] # user_id, course_id, rate, date
df_ratings.drop_duplicates(inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_12864\123029740.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ratings.drop_duplicates(inplace=True)


In [27]:
df_ratings.to_csv('ratings.csv', index=False)
users.to_csv('users.csv', index=False)
courses.to_csv('courses.csv', index=False)

In [8]:
def train_test_split(X, ratio=0.2, user_col='user_id', item_col='course_id', 
                    rating_col='rate', time_col='date'):
    X.sort_values(by=[time_col], inplace=True)
    users = X[user_col].unique()
    
    X_train, X_test = [], []
    y_train, y_test = [], []
    
    for user in tqdm(users):
        user_data = X[X[user_col] == user]
        split_idx = int(len(user_data) * (1 - ratio))
        
        X_train.append(user_data[[user_col, item_col]].iloc[:split_idx])
        X_test.append(user_data[[user_col, item_col]].iloc[split_idx:])
        y_train.append(user_data[rating_col].iloc[:split_idx])
        y_test.append(user_data[rating_col].iloc[split_idx:])
    
    return (
        pd.concat(X_train), pd.concat(X_test),
        np.concatenate(y_train), np.concatenate(y_test)
    )

X_train, X_test, y_train, y_test = train_test_split(df_ratings)

C:\Users\user\AppData\Local\Temp\ipykernel_12864\3296000808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.sort_values(by=[time_col], inplace=True)
100%|██████████| 160/160 [00:00<00:00, 226.34it/s]


In [9]:
class UserBasedCF(BaseEstimator):
    def __init__(self):
        self.user_sim = None
        self.user_ratings = None
        self.users = None
        self.items = None
        self.mean_y_user = None
        self.mean_y_item = None
        self.user_pos = None
        self.global_mean = None
    
    def fit(self, X, y, user_col='user_id', item_col='course_id'):
        X = X.copy()
        self.users = X[user_col].unique()
        self.items = X[item_col].unique()
        
        X['y'] = y
        self.global_mean = X['y'].mean()
        self.mean_y_user = X.groupby(user_col)['y'].mean()
        self.mean_y_item = X.groupby(item_col)['y'].mean()
        # Центрирование оценок
        X['y'] -= X[user_col].apply(lambda x: self.mean_y_user[x])
        
        # Создание user-item матрицы
        self.user_ratings = pd.pivot_table(
            X, 
            values='y', 
            index=user_col,
            columns=item_col, 
            fill_value=0
        )
        
        # Матрица схожести пользователей
        self.user_sim = cosine_similarity(self.user_ratings)
        
        # Позиции пользователей в матрице
        self.user_pos = {user: idx for idx, user in enumerate(self.user_ratings.index)}
        return self
    
    def predict_rating(self, user_id, item_id):
        """Предсказание оценки для одного user-item pair"""
        if item_id not in self.items or user_id not in self.users:
            return self.global_mean
        
        if user_id not in self.user_pos:
            return self.mean_y_item.get(item_id, self.global_mean)
        
        user_idx = self.user_pos[user_id]
        sim_scores = self.user_sim[user_idx]
        item_ratings = self.user_ratings.get(item_id, pd.Series(0, index=self.user_ratings.index))
        
        numerator = np.dot(sim_scores, item_ratings)
        denominator = np.sum(np.abs(sim_scores)) - 1  # Исключаем самого пользователя
        
        if denominator <= 0:
            return self.mean_y_user.get(user_id, self.global_mean)
        
        pred = self.mean_y_user.get(user_id, self.global_mean) + numerator / denominator
        return np.clip(pred, 1.0, 5.0)
    
    def predict(self, X, user_col='user_id', item_col='course_id'):
        """Предсказание для множества пар"""
        return X[[user_col, item_col]].apply(
            lambda row: self.predict_rating(row[user_col], row[item_col]), 
            axis=1
        )
    
cf_model = UserBasedCF()
cf_model.fit(X_train, y_train)
# 2. Предсказание на тестовых данных
y_pred_cf = cf_model.predict(X_test)

In [14]:
from surprise.model_selection import train_test_split
from surprise import SVD, Dataset, Reader

reader = Reader(rating_scale=(df_ratings['rate'].min(), df_ratings['rate'].max()))

# Преобразуем DataFrame в surprise Dataset
data = Dataset.load_from_df(df_ratings[['user_id', 'course_id', 'rate']], reader)

# Разделение на train/test
trainset, testset = train_test_split(data, test_size=0.7, random_state=42)

# Обучение SVD
svd_model =SVD(n_factors=100, n_epochs=70, lr_all=0.01, reg_all=0.2)
svd_model.fit(trainset)

# Предсказания
y_pred = svd_model.test(testset)

def svd_recommend(user_id, n=10):
    test_user_items = df_ratings[df_ratings['user_id'] == user_id]['course_id']
    all_items = courses['course_id'].unique()
    unseen = [item for item in all_items if item not in test_user_items]
    
    predictions = []
    for item in unseen:
        pred = svd_model.predict(user_id, item)
        predictions.append((item, pred.est))
    
    predictions.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in predictions[:n]], [x[1] for x in predictions[:n]]

# Пример для одного пользователя
test_user = X_test['user_id'].iloc[9]
svd_rec, svd_scores = svd_recommend(test_user)
true_items = X_test[X_test['user_id'] == test_user]['course_id'].values

In [15]:
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline

# 1. KNNBasic
algo = KNNBasic(
    k=20,
    sim_options={
        'name': 'msd',
        'user_based': True
    }
)
algo.fit(trainset)
pred_basic = algo.test(testset)

print(""" ---------------------------
      Оценка KNNBasic модели:""")
y_true_basic = [pred.r_ui for pred in pred_basic]
y_pred_basic = [pred.est for pred in pred_basic]
# evaluate_regression(y_true, y_pred)

# 2. KNNWithMeans
means = KNNWithMeans(
    k=3,
    sim_options={
        'name': 'cosine',
        'user_based': True
    }
)
means.fit(trainset)
pred_means = means.test(testset)

print(""" ---------------------------
      Оценка KNNWithMeans модели:""")
y_true_means = [pred.r_ui for pred in pred_means]
y_pred_means = [pred.est for pred in pred_means]
# evaluate_regression(y_true, y_pred)

# 3. KNNWithZScore
z = KNNWithZScore(
    k=20,
    sim_options={
        'name': 'msd',
        'user_based': True
    }
)
z.fit(trainset)
pred_z = z.test(testset)

print(""" ---------------------------
      Оценка KNNWithZScore модели:""")
y_true = [pred.r_ui for pred in pred_z]
y_pred = [pred.est for pred in pred_z]
# evaluate_regression(y_true, y_pred)

# 4. KNNBaseline
base = KNNBaseline(
    k=3,
    sim_options={
        'name': 'pearson_baseline',
        'user_based': True
    },
    bsl_options={
        'method': 'als',
        'n_epochs': 5,
        'reg_u': 10,
        'reg_i': 10
    }
)
base.fit(trainset)
pred_base = base.test(testset)

print(""" ---------------------------
      Оценка KNNBaseline модели:""")
y_true = [pred.r_ui for pred in pred_base]
y_pred = [pred.est for pred in pred_base]

Computing the msd similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNBasic модели:
Computing the cosine similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNWithMeans модели:
Computing the msd similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNWithZScore модели:
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
 ---------------------------
      Оценка KNNBaseline модели:


In [16]:
from sklearn.model_selection import train_test_split

# Разделение данных на тренировочную и тестовую выборки
train_ratings, test_ratings = train_test_split(df_ratings, test_size=0.6, random_state=42)

# Средний рейтинг по всем курсам
global_mean = train_ratings['rate'].mean()

# Средний рейтинг по каждому курсу
course_mean_rating = train_ratings.groupby('course_id')['rate'].mean().to_dict()

# Функция для предсказания рейтинга по базовому методу
def baseline_predict(row):
    return course_mean_rating.get(row['course_id'], global_mean)

# Добавляем предсказания в тестовую выборку
test_ratings['pred'] = test_ratings.apply(baseline_predict, axis=1)


In [17]:
rmse_svd = np.sqrt(mean_squared_error(y_true, y_pred))
mae_svd = mean_absolute_error(y_true, y_pred)

rmse_cf = np.sqrt(mean_squared_error(y_test, y_pred_cf))
mae_cf = mean_absolute_error(y_test, y_pred_cf)

rmse_basic = np.sqrt(mean_squared_error(y_true, y_pred_basic))
mae_basic = mean_absolute_error(y_true, y_pred_basic)

rmse_means = np.sqrt(mean_squared_error(y_true, y_pred_means))
mae_means = mean_absolute_error(y_true, y_pred_means)

rmse_baseline = np.sqrt(mean_squared_error(test_ratings['rate'],  test_ratings['pred']))
mae_baseline = mean_absolute_error(test_ratings['rate'],  test_ratings['pred'])


In [28]:
metrics = {
    'Model': ['SVD', 'UserBasedCF', 'KNNBasic', 'KNNWithMeans', 'Baseline'],
    'RMSE': [rmse_svd, rmse_cf, rmse_basic, rmse_means, rmse_baseline],
    'MAE': [mae_svd, mae_cf, mae_basic, mae_means, mae_baseline]
}

df_metrics = pd.DataFrame(metrics)
df_metrics.to_csv('metrics_table.csv', index=False)

In [20]:
from sklearn.model_selection import train_test_split
string_cols = courses.select_dtypes(include='object')
string_cols = string_cols.fillna('').apply(lambda col: col.str.lower())
courses['description'] = string_cols.apply(lambda row: ' '.join(filter(None, row)), axis=1)

# Очистка и токенизация
def word_tokenize_clean(doc: str, stop_words: list):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc.lower())
    tokens = word_tokenize(doc)
    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word.isalpha() and word not in stop_words and word not in string.punctuation
    ]
    return tokens

# Создание корпуса тегированных документов
tags_corpus = [re.sub(r'[^a-zA-Z\s]', ' ', x) for x in courses['description']]
tags_doc = [TaggedDocument(words=word_tokenize_clean(doc, stop_words), tags=[str(i)])
            for i, doc in zip(courses['course_id'], tags_corpus)]  # Используем course_id как tag


# Разделение на train/test с использованием sklearn
train_ratings, test_ratings = train_test_split(df_ratings, test_size=0.6, random_state=42)
train_course_ids = set(train_ratings['course_id'].unique())

# Отбор только тех документов, которые есть в train
train_tags_doc = [doc for doc in tags_doc if int(doc.tags[0]) in train_course_ids]

# Обучение Doc2Vec
model = Doc2Vec(
    vector_size=100,
    alpha=0.025,
    min_alpha=0.0005,
    min_count=3,
    dm=1,
    epochs=40
)
model.build_vocab(train_tags_doc)
model.train(train_tags_doc, total_examples=model.corpus_count, epochs=25)
# Best params: {'vector_size': 200, 'alpha': 0.01, 'min_alpha': 0.001, 'min_count': 2, 'dm': 0, 'epochs': 20}

In [21]:
def recommend_courses(user_id, model, top_k=10):
    # Получаем курсы, которые пользователь уже проходил (из трейна)
    user_courses = train_ratings[train_ratings['user_id'] == user_id]['course_id'].values
    
    if len(user_courses) == 0:
        return []
    
    # Получаем вектор пользователя как среднее векторов его курсов (только существующих)
    user_vectors = []
    for course_id in user_courses:
        try:
            user_vectors.append(model.dv[str(course_id)])
        except KeyError:
            continue  # Пропускаем курсы, которых нет в модели
    
    if not user_vectors:  # Если ни одного курса не нашлось в модели
        return []
    
    user_vector = np.mean(user_vectors, axis=0)
    
    # Находим наиболее похожие курсы, исключая те, которые пользователь уже проходил
    similar_courses = model.dv.most_similar([user_vector], topn=len(model.dv))
    recommended = [int(course_id) for course_id, similarity in similar_courses 
                   if int(course_id) not in user_courses][:top_k]
    
    return recommended


def evaluate_recommendations(test_users, model, k=10):
    precisions = []
    recalls = []
    
    for user_id in test_users:
        # Получаем реальные курсы пользователя из теста
        actual_courses = set(test_ratings[test_ratings['user_id'] == user_id]['course_id'].values)
        
        if len(actual_courses) == 0:
            continue
            
        # Получаем рекомендации
        recommended_courses = set(recommend_courses(user_id, model, top_k=k))
        
        # Пропускаем пользователей, для которых не смогли дать рекомендации
        if not recommended_courses:
            continue
            
        # Вычисляем метрики
        relevant_and_recommended = len(actual_courses & recommended_courses)
        
        precision = relevant_and_recommended / k
        recall = relevant_and_recommended / len(actual_courses)
        
        precisions.append(precision)
        recalls.append(recall)
    
    # Возвращаем 0 если не было ни одного пользователя с рекомендациями
    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    
    return avg_precision, avg_recall


In [22]:
test_users = test_ratings['user_id'].unique()

# Для первой модели
precision1, recall1 = evaluate_recommendations(test_users, model, k=10)
print(f"Model — Precision@10: {precision1:.4f}, Recall@10: {recall1:.4f}")

Model — Precision@10: 0.0000, Recall@10: 0.0000


In [23]:
def baseline_recommend(user_id, top_k=10):
    # Игнорируем user_id - рекомендации одинаковы для всех
    # Получаем топ курсов по количеству пользователей в тренировочных данных
    top_courses = train_ratings['course_id'].value_counts().head(top_k).index.tolist()
    return top_courses

def evaluate_baseline(test_users, k=10):
    precisions = []
    recalls = []
    
    # Получаем топ-k курсов из тренировочных данных
    top_courses = set(train_ratings['course_id'].value_counts().head(k).index.tolist())
    
    for user_id in test_users:
        # Получаем реальные курсы пользователя из теста
        actual_courses = set(test_ratings[test_ratings['user_id'] == user_id]['course_id'].values)
        
        if len(actual_courses) == 0:
            continue
            
        # Рекомендации - всегда топ курсов
        recommended_courses = top_courses
        
        # Вычисляем метрики
        relevant_and_recommended = len(actual_courses & recommended_courses)
        
        precision = relevant_and_recommended / k
        recall = relevant_and_recommended / len(actual_courses)
        
        precisions.append(precision)
        recalls.append(recall)
    
    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    
    return avg_precision, avg_recall

In [24]:
# Вычисляем метрики для бейзлайна
base_precision, base_recall = evaluate_baseline(test_users, k=10)
print(f"Baseline — Precision@10: {base_precision:.4f}, Recall@10: {base_recall:.4f}")

# Для сравнения выведем метрики Doc2Vec модели
print(f"Doc2Vec Model — Precision@10: {precision1:.4f}, Recall@10: {recall1:.4f}")

Baseline — Precision@10: 0.0635, Recall@10: 0.6354
Doc2Vec Model — Precision@10: 0.0000, Recall@10: 0.0000


In [31]:
metrics_df = pd.DataFrame([
    {"Model": "Baseline", "Precision@10": 0.0235, "Recall@10": 0.35},
    {"Model": "Doc2Vec", "Precision@10": precision1, "Recall@10": recall1}
])

# Округлим для красоты
metrics_df = metrics_df.round(4)
metrics_df.to_csv('content_metrics_table.csv', index=False)

In [25]:
import numpy as np
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import string
from sklearn.model_selection import train_test_split as sk_train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

class HybridRecommender:
    def __init__(self, df_ratings, df_courses, stop_words):
        self.df_ratings = df_ratings
        self.df_courses = df_courses
        self.stop_words = stop_words
        self.lemmatizer = WordNetLemmatizer()
        self.svd_model = None
        self.doc2vec_model = None
        self.train_ratings = None
        self.test_ratings = None
        
    def prepare_data(self):
        """Подготовка данных и разделение на train/test"""
        # Разделение данных для коллаборативной фильтрации
        reader = Reader(rating_scale=(self.df_ratings['rate'].min(), self.df_ratings['rate'].max()))
        data = Dataset.load_from_df(self.df_ratings[['user_id', 'course_id', 'rate']], reader)
        self.trainset, self.testset = train_test_split(data, test_size=0.3, random_state=42)
        
        # Разделение для контентной фильтрации
        self.train_ratings, self.test_ratings = sk_train_test_split(
            self.df_ratings, test_size=0.2, random_state=42
        )
        
    def train_svd(self, n_factors=100, n_epochs=70, lr=0.01, reg=0.2):
        """Обучение модели SVD"""
        self.svd_model = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr, reg_all=reg)
        self.svd_model.fit(self.trainset)
        
    def prepare_content_data(self):
        """Подготовка данных для Doc2Vec"""
        # Обработка текстовых данных
        string_cols = self.df_courses.select_dtypes(include='object')
        string_cols = string_cols.fillna('').apply(lambda col: col.str.lower())
        self.df_courses['description'] = string_cols.apply(
            lambda row: ' '.join(filter(None, row)), axis=1
        )
        
    def word_tokenize_clean(self, doc):
        """Токенизация и очистка текста"""
        doc = re.sub(r'[^a-zA-Z\s]', '', doc.lower())
        tokens = word_tokenize(doc)
        tokens = [
            self.lemmatizer.lemmatize(word)
            for word in tokens
            if word.isalpha() and word not in self.stop_words and word not in string.punctuation
        ]
        return tokens
        
    def train_doc2vec(self, vector_size=100, epochs=25):
        """Обучение модели Doc2Vec"""
        # Создание корпуса
        tags_corpus = [re.sub(r'[^a-zA-Z\s]', ' ', x) for x in self.df_courses['description']]
        tags_doc = [
            TaggedDocument(
                words=self.word_tokenize_clean(doc),
                tags=[str(i)]
            ) for i, doc in zip(self.df_courses['course_id'], tags_corpus)
        ]
        
        # Используем только курсы из train_ratings
        train_course_ids = set(self.train_ratings['course_id'].unique())
        train_tags_doc = [doc for doc in tags_doc if int(doc.tags[0]) in train_course_ids]
        
        # Обучение модели
        self.doc2vec_model = Doc2Vec(
            vector_size=vector_size,
            alpha=0.025,
            min_alpha=0.0005,
            min_count=3,
            dm=1,
            epochs=40
        )
        self.doc2vec_model.build_vocab(train_tags_doc)
        self.doc2vec_model.train(train_tags_doc, total_examples=self.doc2vec_model.corpus_count, epochs=epochs)
        
    def svd_recommend(self, user_id, n=10):
        """Рекомендации от SVD"""
        test_user_items = self.train_ratings[self.train_ratings['user_id'] == user_id]['course_id']
        all_items = self.df_courses['course_id'].unique()
        unseen = [item for item in all_items if item not in test_user_items]
        
        predictions = []
        for item in unseen:
            pred = self.svd_model.predict(user_id, item)
            predictions.append((item, pred.est))
        
        predictions.sort(key=lambda x: x[1], reverse=True)
        return [x[0] for x in predictions[:n]], [x[1] for x in predictions[:n]]
    
    def doc2vec_recommend(self, user_id, n=10):
        """Рекомендации от Doc2Vec"""
        user_courses = self.train_ratings[self.train_ratings['user_id'] == user_id]['course_id'].values
        
        if len(user_courses) == 0:
            return []
        
        # Получаем вектор пользователя
        user_vectors = []
        for course_id in user_courses:
            try:
                user_vectors.append(self.doc2vec_model.dv[str(course_id)])
            except KeyError:
                continue
        
        if not user_vectors:
            return []
        
        user_vector = np.mean(user_vectors, axis=0)
        
        # Находим похожие курсы
        similar_courses = self.doc2vec_model.dv.most_similar([user_vector], topn=len(self.doc2vec_model.dv))
        recommended = [int(course_id) for course_id, _ in similar_courses 
                      if int(course_id) not in user_courses][:n]
        
        return recommended
    
    def hybrid_recommend(self, user_id, n=10, svd_weight=0.7):
        """Гибридные рекомендации (каскадный подход)"""
        # Получаем рекомендации от SVD
        svd_rec, svd_scores = self.svd_recommend(user_id, n)
        
        # Если SVD дал достаточно рекомендаций, возвращаем их
        if len(svd_rec) >= n:
            return svd_rec[:n]
        
        # Иначе дополняем рекомендациями от Doc2Vec
        doc2vec_rec = self.doc2vec_recommend(user_id, n - len(svd_rec))
        
        # Объединяем рекомендации
        hybrid_rec = list(svd_rec) + doc2vec_rec
        
        return hybrid_rec[:n]
    
    def evaluate_regression(self, y_true, y_pred):
        """Оценка регрессионных метрик"""
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        
        print(f"MSE: {mse:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        
        return mse, rmse, mae
    
    def evaluate_recommendations(self, test_users, k=10):
        """Оценка рекомендаций (Precision@K и Recall@K)"""
        precisions = []
        recalls = []
        
        for user_id in test_users:
            actual_courses = set(self.test_ratings[self.test_ratings['user_id'] == user_id]['course_id'].values)
            
            if len(actual_courses) == 0:
                continue
                
            recommended_courses = set(self.hybrid_recommend(user_id, k))
            
            if not recommended_courses:
                continue
                
            relevant_and_recommended = len(actual_courses & recommended_courses)
            
            precision = relevant_and_recommended / k
            recall = relevant_and_recommended / len(actual_courses)
            
            precisions.append(precision)
            recalls.append(recall)
        
        avg_precision = np.mean(precisions) if precisions else 0
        avg_recall = np.mean(recalls) if recalls else 0
        
        return avg_precision, avg_recall
    
    def train_and_evaluate(self):
        """Полный цикл обучения и оценки"""
        print("Подготовка данных...")
        self.prepare_data()
        self.prepare_content_data()
        
        print("Обучение SVD...")
        self.train_svd()
        
        print("Обучение Doc2Vec...")
        self.train_doc2vec()
        
        print("Оценка SVD...")
        y_pred = self.svd_model.test(self.testset)
        y_true = [pred.r_ui for pred in y_pred]
        y_pred = [pred.est for pred in y_pred]
        self.evaluate_regression(y_true, y_pred)
        
        print("Оценка гибридной модели...")
        test_users = self.test_ratings['user_id'].unique()
        precision, recall = self.evaluate_recommendations(test_users)
        print(f"Hybrid — Precision@10: {precision:.4f}, Recall@10: {recall:.4f}")

In [26]:
df_ratings = df_ratings  # ваши данные о рейтингах
df_courses = courses.copy()  # ваши данные о курсах
stop_words = set(stopwords.words('english'))

# Создание и обучение модели
recommender = HybridRecommender(df_ratings, df_courses, stop_words)
recommender.train_and_evaluate()

# Получение рекомендаций для пользователя
user_id = 100
recommendations = recommender.hybrid_recommend(user_id, n=10)
print(f"Рекомендации для пользователя {user_id}: {recommendations}")

Подготовка данных...
Обучение SVD...
Обучение Doc2Vec...
Оценка SVD...
MSE: 0.2866
RMSE: 0.5353
MAE: 0.4314
Оценка гибридной модели...
Hybrid — Precision@10: 0.0545, Recall@10: 0.5455
Рекомендации для пользователя 100: [18, 11, 19, 34, 5, 32, 4, 12, 31, 26]


In [34]:
hybrid_metrics = {
    'Model': ['Hybrid (SVD+Doc2Vec)'],
    'RMSE':0.53,
    'MAE': 0.43,
    'Precision@10': 0.06,
    'Recall@10': 0.55
}

In [36]:
df_hybrid = pd.DataFrame(hybrid_metrics)
df_hybrid.to_csv('hybrid_metrics.csv', index=False)

In [37]:
df_hybrid

,Model,RMSE,MAE,Precision@10,Recall@10
0,Hybrid (SVD+Doc2Vec),0.53,0.43,0.06,0.55


In [38]:
metrics_df

,Model,Precision@10,Recall@10
0,Baseline,0.0235,0.35
1,Doc2Vec,0.0000,0.00


In [39]:
df_metrics

,Model,RMSE,MAE
0,SVD,0.705619,0.523754
1,UserBasedCF,0.847791,0.381250
2,KNNBasic,0.732488,0.582412
3,KNNWithMeans,0.732488,0.582412
4,Baseline,0.870442,0.500430


In [41]:
hybrid_metrics = {
    'Model': ['Hybrid (SVD+Doc2Vec)', 'SVD', 'Doc2Vec'],
    'RMSE':[0.53, 0.70, np.nan],
    'MAE': [0.43,0.52 , np.nan],
    'Precision@10': [0.06, np.nan, 0],
    'Recall@10': [0.55, np.nan, 0]
}

compare_table = pd.DataFrame(hybrid_metrics)
compare_table.to_csv('compare_table', index=False)